This is a port from Python of the Wedding Table Assignment example from [PuLP](https://github.com/coin-or/pulp/blob/master/examples/wedding.py).

In [9]:
using JuMP, Cbc
using Combinatorics

This is an example of a **set partitioning** problem. We have a _universe_ of guests and subsets of tables that have to _exactly_ cover our universe, i.e. the guests.

Silly function to calculate "happiness" of a table, determined by how close the guests are (by their letter):

In [14]:
function happiness(table)
    """
    Find the happiness of the table
    - by calculating the maximum distance between the letters
    """
    return abs(Int(table[1][1]) - Int(table[end][1]))
end

happiness (generic function with 1 method)

In [15]:
max_tables = 5
max_table_size = 4
guests = ["A" "B" "C" "D" "E" "F" "G" "I" "J" "K" "L" "M" "N" "O" "P" "Q" "R"]

1×17 Array{String,2}:
 "A"  "B"  "C"  "D"  "E"  "F"  "G"  "I"  …  "L"  "M"  "N"  "O"  "P"  "Q"  "R"

In [16]:
table_combos = [collect(combinations(guests, t)) for t in 1:max_table_size];

In [17]:
table_combos[end][end]    

4-element Array{String,1}:
 "O"
 "P"
 "Q"
 "R"

In [18]:
# this seems suboptimal, there's probably a better to flatten out these combinations
possible_tables = []
for i in 1:length(table_combos)
    append!(possible_tables, table_combos[i])
end

In [19]:
length(possible_tables)

3213

In [20]:
m = Model(solver=CbcSolver())
num_possible_tables = length(possible_tables)
idx_possible_tables = 1:num_possible_tables

@variable(m, table_assignment[idx_possible_tables], Bin)

# Objective: maximize happiness = minimize happiness value
@objective(m, Min, sum([happiness(possible_tables[t]) * table_assignment[t] for t in idx_possible_tables]))

@constraint(m, sum([table_assignment[t] for t in idx_possible_tables]) <= max_tables)


# because this is a set partitioning problem, we enforce a strict equality constraint 
# - i.e. every guest has to be seated
for guest in guests
    @constraint(m, sum([table_assignment[t] for t in idx_possible_tables if guest in possible_tables[t]]) == 1)
end

;

┌ Warning: Deprecated syntax `implicit assignment to global variable `#3###368``.
│ Use `global #3###368` instead.
└ @ nothing none:0
┌ Warning: Deprecated syntax `implicit assignment to global variable `#3###368``.
│ Use `global #3###368` instead.
└ @ nothing none:0


In [21]:
status = solve(m)

println("Objective value: ", getobjectivevalue(m))

Objective value: 12.0


In [22]:
table_assignment

table_assignment[i] ∈ {0,1} ∀ i ∈ {1,2,…,3212,3213}

In [23]:
[("Table: ", possible_tables[i], "Happiness: ", happiness(possible_tables[i])) for i=1:length(table_assignment) if getvalue(table_assignment[i]) == 1 ]

5-element Array{Tuple{String,Array{String,1},String,Int64},1}:
 ("Table: ", ["Q", "R"], "Happiness: ", 1)          
 ("Table: ", ["E", "F", "G"], "Happiness: ", 2)     
 ("Table: ", ["A", "B", "C", "D"], "Happiness: ", 3)
 ("Table: ", ["I", "J", "K", "L"], "Happiness: ", 3)
 ("Table: ", ["M", "N", "O", "P"], "Happiness: ", 3)